<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/BlackScholesMonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Monte Carlo Pricing in the Black Scholes Model**

This codebook shows how to use Monte Carlo to derive option prices in the Black-Scholes model. We will revisit the problems from the previous homeworks.

Again, we have to pull ib functionality from libraries. The most common ones for us are [pandas](https://pandas.pydata.org/), the data analysis library and math and [numpy](https://numpy.org/), which are mathematical libraries. And of course we have to import Quantlib, which we just installed (many packages such as pandas and numpy are natively installed in the Colab environment).

In [ ]:
import pandas as pd
import numpy as np

Let's grab the parameters from Homework 5-2

In [ ]:
S_0 = 3714.24
K = 3714.24
sigma = 0.165
r = 0.001
T = 1

And let's calculate a Monte Carlo price for a European Call

In [ ]:
np.random.seed(42)
normals = np.random.randn(10000)
S = S_0 * np.exp((r-0.5 * sigma * sigma) * T * np.ones(10000) + np.sqrt(T) * sigma * normals)
Payoff = np.zeros(10000)
for i in range(10000):
  Payoff[i] = np.max([K-S[i],0])
Price = np.exp(-r*T) * Payoff.mean()
print(Price)

243.22781286497388


Similarly, let's calculate the Asian option price from part 5-3. Here we have to simulate the stock price across the entire year.

In [ ]:
numbdays = 256
np.random.seed(42)
pricepaths = np.zeros((10000,numbdays+1))
for i in range(10000):
  pricepaths[i,0] = S_0
  for t in range (numbdays):
    pricepaths[i,t+1] = pricepaths[i,t] * np.exp((r-0.5 * sigma * sigma) * T/numbdays + np.sqrt(T/numbdays) * sigma * np.random.randn(1))
pathaverages = np.mean(pricepaths,axis=1)
for i in range(10000):
  Payoff[i] = np.max([K-pathaverages[i],0])
Price2 = np.exp(-r*T) * Payoff.mean()
print(Price2)

138.61164979041922


Let's try two stocks. Again, let's collect the parameters:

In [ ]:
sigma1 = 0.3095
sigma2 = 0.2993
rho = 0.7094
r = 0.02
T = 1
q1 = 1.00
q2 = 2.65
numbdays = 252
S01 = 40.68
S02 = 15.42

Let's take two approaches. First, let's do it in one shot. Let's generate correlated normals.

In [ ]:
np.random.seed(42)
normals1 = np.random.randn(10000)
normalsHelp = np.random.randn(10000)
normals2 = rho * normals1 + np.sqrt(1 - rho * rho) * normalsHelp
S1 = S01 * np.exp((r-0.5 * sigma1 * sigma1) * T * np.ones(10000) + np.sqrt(T) * sigma1 * normals1)
S2 = S02 * np.exp((r-0.5 * sigma2 * sigma2) * T * np.ones(10000) + np.sqrt(T) * sigma2 * normals2)
Payoff = np.zeros(10000)
for i in range(10000):
  Payoff[i] = np.max([q1*S1[i],q2*S2[i]])
Price = np.exp(-r*T) * Payoff.mean()
print(Price)

44.59813646198227


Then, let's simulate the entire paths similar to above:

In [ ]:
numbdays = 256
np.random.seed(42)
pricepaths1 = np.zeros((10000,numbdays+1))
pricepaths2 = np.zeros((10000,numbdays+1))
for i in range(10000):
  pricepaths1[i,0] = S01
  pricepaths2[i,0] = S02
  for t in range (numbdays):
    normal1 = np.random.randn(1)
    normalHelp = np.random.randn(1)
    normal2 = rho * normal1 + np.sqrt(1 - rho * rho) * normalHelp
    pricepaths1[i,t+1] = pricepaths1[i,t] * np.exp((r-0.5 * sigma1 * sigma1) * T/numbdays + np.sqrt(T/numbdays) * sigma1 * normal1)
    pricepaths2[i,t+1] = pricepaths2[i,t] * np.exp((r-0.5 * sigma2 * sigma2) * T/numbdays + np.sqrt(T/numbdays) * sigma2 * normal2)
  Payoff[i] = np.max([q1*pricepaths1[i,numbdays],q2*pricepaths2[i,numbdays]])
Price = np.exp(-r*T) * Payoff.mean()
print(Price)

44.39814898986675
